# Initialization:

In [4]:
import torch
import dqc
import dqc.xc
import dqc.utils
from importlib import reload 
import DQCAdapter as evg

In [5]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a_par, p_par):
        super().__init__()
        self.a_par = a_par
        self.p_par = p_par
        self.number_of_parameters = 2

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a_par * rho ** self.p_par
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p_par
            elif number_of_parameter == 1: # parameter p
                return self.a_par * torch.log(rho) * rho ** self.p_par

In [6]:
a_par = torch.nn.Parameter(torch.tensor(1.0, dtype=torch.double))
p_par = torch.nn.Parameter(torch.tensor(2.0, dtype=torch.double))
myxc = MyLDAX(a_par, p_par)

In [7]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

mol = dqc.Mol(moldesc=water, basis="6-31G")
qc = dqc.KS(mol, xc=myxc).run()
ene = qc.energy()
ene.detach()

The 6-31G basis for atomz 8 does not exist, but we will download it
Downloaded to /home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/dqc/api/.database/6-31g/08.gaussian94
The 6-31G basis for atomz 1 does not exist, but we will download it
Downloaded to /home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/dqc/api/.database/6-31g/01.gaussian94


tensor(-38.2050, dtype=torch.float64)

In [24]:
reload(evg);
adapter = evg.DQCAdapter(qc)

partial g: tensor([ 9.9156, 19.0426], dtype=torch.float64)
adjoint: 15.576124348236934
partial f: 46.60346291417592
product: tensor([8.0394e-15, 3.9445e-14], dtype=torch.float64)


In [6]:
fockian = adapter.get_fockian()

In [7]:
coefficients = adapter.get_orbital_coefficients()

In [8]:
orbital_energies = adapter.get_orbital_energies()

In [9]:
occupancy = adapter.get_orbital_occupancy()

In [10]:
number_of_occupied_orbitals = adapter.get_number_of_occupied_orbitals()

# Calculating adjoint derivatives:

## Derivatives from energy:

### Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}}$

#### The first way to calculate derivative from energy with respect to coefficients:
Every type of orbitals can be used

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\textbf{C}\textbf{F}$$
$\textbf{f}$ here is matrix: $f_{ab}=\delta_{ab}f_a$

In [11]:
dE_wrt_dC_first_way = 2 * torch.einsum("b,ib,ij->bj", occupancy, coefficients, fockian)

In [12]:
dE_by_dC = 2 * occupancy.view((5,1)) * coefficients.t() @ fockian
torch.dist(dE_by_dC, dE_wrt_dC_first_way)

tensor(0., dtype=torch.float64)

In [14]:
%timeit occupancy.view((5,1)) * coefficients.t() @ fockian

7.77 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
%timeit torch.einsum("b,ib,ij->bj", occupancy, coefficients, fockian)

20.8 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### The second way to calculate derivative from energy with respect to coefficients
Only canonical (i.e. eigenfunctions of fockian) orbitals can be used

from the other hand, 
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial C_{bj}} = 2f_b \sum_i C_{bi}F_{ij} = 2f_b \epsilon_b C_{bj}$$
so
$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \textbf{C}} = 2\textbf{f}\epsilon\textbf{C}$$
$\epsilon$ here is matrix: $\epsilon_{ab}=\delta_{ab}\epsilon_a$

In [16]:
dE_wrt_dC_second_way = 2 * torch.einsum("b,b,jb->bj", occupancy, orbital_energies, coefficients)

Check that both ways lead to the same results:

In [17]:
print(torch.linalg.matrix_norm(dE_wrt_dC_first_way - dE_wrt_dC_second_way))

tensor(8.4341e-14, dtype=torch.float64)


### Calculating $\frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\epsilon}}$

$$\frac{\partial E[\rho](\vec{\theta})}{\partial \epsilon_{b}} = 0$$ for all $\epsilon_b$. So:

In [34]:
dE_wrt_depsilon = torch.zeros(number_of_occupied_orbitals)

## Derivatives from normalization equations:

### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \textbf{C}}$

$$\frac{\partial r_{a}(\textbf{C})}{\partial C_{ck}} = 2\delta_{ac}C_{ck}$$


In [18]:
occupied_orbitals_kronecker = torch.eye(number_of_occupied_orbitals)

In [19]:
dnorm_wrt_dC = 2 * torch.einsum("ac,kc->kac", occupied_orbitals_kronecker, coefficients)

### Calculating $\frac{\partial \vec{r}(\textbf{C})}{\partial \vec{\epsilon}}$

$$\frac{\partial r_{a}(\textbf{C})}{\partial \epsilon_b} = 0$$


In [20]:
dnorm_wrt_depsilon = torch.zeros((number_of_occupied_orbitals, number_of_occupied_orbitals))

## Derivatives from Roothan equations:

### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \textbf{C}}$

$$G_{Cou, ijkl} = \iint b_i(\vec{r}) \frac{b_k(\vec{r'})b_l(\vec{r'})}{|\vec{r}-\vec{r'}|} b_j(\vec{r})d\vec{r'}d\vec{r}$$
$$G_{XC, ijkl} = 
\int b_i(\vec{r}) b_k(\vec{r}) \frac{\partial V_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \rho(\vec{r})}b_l(\vec{r})b_j(\vec{r})d\vec{r}$$
$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial C_{ck}} = 
(F_{ik}[\rho](\vec{\theta}) - \epsilon_c\delta_{ik})\delta_{ac} +
2f_c\sum_j \sum_{l}C_{cl}\left(G_{Cou,ijkl} + G_{XC,ijkl}\right)C_{aj}
$$

First term:

In [21]:
number_of_all_orbitals = adapter.get_number_of_all_orbitals()
all_orbitals_kronecker = torch.eye(number_of_all_orbitals)
dRoothan_wrt_dC_first_term = torch.einsum("ik,ac->iakc", fockian, occupied_orbitals_kronecker)
dRoothan_wrt_dC_first_term -= torch.einsum("c,ik,ac->iakc", orbital_energies, all_orbitals_kronecker, occupied_orbitals_kronecker)

Second term:
(NB: fourDtensor loses symmetry of XC-tensor):

In [22]:
fourDtensor = adapter.get_four_center_elrep_tensor() + adapter.get_four_center_xc_tensor()
dRoothan_wrt_dC_second_term = 2 * torch.einsum("c,ijkl,lc,ja->iakc", occupancy, fourDtensor, coefficients, coefficients)

The sum:

In [23]:
dRoothan_wrt_dC = dRoothan_wrt_dC_first_term + dRoothan_wrt_dC_second_term

### Calculating $\frac{\partial \textbf{r}(\textbf{C};\;\vec{\theta})}{\partial \vec{\epsilon}}$


$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \epsilon_{c}} = -\delta_{ac}C_{ai}$$

In [32]:
dRoothan_wrt_depsilon = -1 * torch.einsum("ac,ia->iac", occupied_orbitals_kronecker, coefficients)

# Calculate adjoint vector

## Concatenate tensors:

### $\frac{\partial E}{\partial \vec{X}}$

$$\left( \frac{\partial E}{\partial \vec{X}} \right)_{N_{ao}j+c} =  \begin{cases}
   \frac{\partial E}{\partial С_{cj}} &\text{if $0 \le j < N_{orb}$} \\
   \frac{\partial E}{\partial \epsilon_{c}} &\text{if $j = N_{orb}$}
 \end{cases}$$

In [45]:
print(dE_wrt_depsilon.unsqueeze(-1).size())
print(dE_wrt_dC_second_way.size())
dE_wrt_dX = torch.cat((dE_wrt_dC_second_way, dE_wrt_depsilon.unsqueeze(-1)), 1)
print(dE_wrt_dX.size())
dE_wrt_dX = dE_wrt_dX.t().reshape(-1,)
print(dE_wrt_dX.size())

torch.Size([5, 1])
torch.Size([5, 13])
torch.Size([5, 14])
torch.Size([70])


Check it:

In [23]:
for j in range(number_of_all_orbitals + 1):
    for c in range(number_of_occupied_orbitals):
        index = number_of_occupied_orbitals * j + c
        if (j < number_of_all_orbitals):
            assert(dE_wrt_dX[index] == dE_wrt_dC_second_way[c][j])
        else:
            assert(dE_wrt_dX[index] == dE_wrt_depsilon[c])

### $\frac{\partial \vec{Y}}{\partial \vec{X}}$

$$\left( \frac{\partial \vec{Y}}{\partial \vec{X}} \right)_{N_{ao}i+a,N_{ao}j+c} =  \begin{cases}
   \frac{\partial r_{ia}}{\partial С_{cj}} &\text{if $0 \le i < N_{orb}$ and $0 \le j < N_{orb}$} \\
   \frac{\partial r_{ia}}{\partial \epsilon_{c}} &\text{if $0 \le i < N_{orb}$ and $j = N_{orb}$} \\
   \frac{\partial r_{a}}{\partial С_{cj}} &\text{if $i = N_{orb}$ and $0 \le j < N_{orb}$} \\
   \frac{\partial r_{a}}{\partial \epsilon_{c}} &\text{if $i = N_{orb}$ and $j = N_{orb}$}
 \end{cases}$$

In [46]:
nao_norb_product = number_of_all_orbitals * number_of_occupied_orbitals
upper_left = dRoothan_wrt_dC.reshape((nao_norb_product,nao_norb_product))
print(dRoothan_wrt_dC.size(), "=>", upper_left.size())
upper_right = dRoothan_wrt_depsilon.reshape((nao_norb_product, number_of_occupied_orbitals))
print(dRoothan_wrt_depsilon.size(), "=>", upper_right.size())
upper = torch.cat((upper_left, upper_right), 1)
print(upper_left.size(), "+", upper_right.size(), "=", upper.size())

torch.Size([13, 5, 13, 5]) => torch.Size([65, 65])
torch.Size([13, 5, 5]) => torch.Size([65, 5])
torch.Size([65, 65]) + torch.Size([65, 5]) = torch.Size([65, 70])


In [47]:
lower_left = dnorm_wrt_dC.reshape(nao_norb_product, number_of_occupied_orbitals).t()
print(dnorm_wrt_dC.size(), "=>", lower_left.size())
lower = torch.cat((lower_left, dnorm_wrt_depsilon), 1)
print(lower_left.size(), "+", dnorm_wrt_depsilon.size(), "=", lower.size())

torch.Size([13, 5, 5]) => torch.Size([5, 65])
torch.Size([5, 65]) + torch.Size([5, 5]) = torch.Size([5, 70])


In [49]:
dY_wrt_dX = torch.cat((upper, lower), 0)
print(upper.size(), "+", lower.size(), "=", dY_wrt_dX.size())

torch.Size([65, 70]) + torch.Size([5, 70]) = torch.Size([70, 70])


Check it:

In [27]:
for i in range(number_of_all_orbitals + 1):
    for a in range(number_of_occupied_orbitals):
        for j in range(number_of_all_orbitals + 1):
            for c in range(number_of_occupied_orbitals):
                index = (number_of_occupied_orbitals * i + a, number_of_occupied_orbitals * j + c)
                if (i < number_of_all_orbitals):
                    if (j < number_of_all_orbitals):
                        assert(dY_wrt_dX[index[0]][index[1]] == dRoothan_wrt_dC[i][a][j][c]),(i,a,j,c)
                    else:
                        assert(dY_wrt_dX[index[0]][index[1]] == dRoothan_wrt_depsilon[i][a][c]),(i,a,c)
                else:
                    if (j < number_of_all_orbitals):
                        assert(dY_wrt_dX[index[0]][index[1]] == dnorm_wrt_dC[j][a][c]),(j,a,c)
                    else:
                        assert(dY_wrt_dX[index[0]][index[1]] == dnorm_wrt_depsilon[a][c]),(a,c)

## Find inverse matrix of $\frac{\partial \vec{Y}}{\partial \vec{X}}$

In [50]:
dY_wrt_dX_inverse = torch.linalg.inv(dY_wrt_dX)
print(dY_wrt_dX_inverse.size())

torch.Size([70, 70])


In [51]:
torch.linalg.matrix_norm((dY_wrt_dX_inverse @ dY_wrt_dX) - torch.eye(dY_wrt_dX.shape[0]))
torch.linalg.matrix_norm((dY_wrt_dX @ dY_wrt_dX_inverse) - torch.eye(dY_wrt_dX.shape[0]))

tensor(7.8064e-13, dtype=torch.float64)

## Find adjoint vector:

In [61]:
adjoint_vector = torch.matmul(dY_wrt_dX_inverse, dE_wrt_dX)
print(adjoint_vector.size())

torch.Size([70])


# Calculating derivatives with respect to parameters

## Derivative from energy: $\frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\theta}}$

$$ \frac{\partial E[\rho](\vec{\theta})}{\partial \vec{\theta}} = 
 \int  \frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}d\vec{r} 
$$
so we just can put $\frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}$ instead of $\rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})$ to DQC calculation.

In [67]:
dE_wrt_dtheta = adapter.get_derivative_of_exc_wrt_theta()
print(dE_wrt_dtheta.size())

torch.Size([2])


## Derivative from normalization equations:

$$\frac{\partial r_{a}(\textbf{C})}{\partial\vec{\theta}}= 0$$ so

In [66]:
number_of_parameters = adapter.get_number_of_parameters()
dnorm_wrt_dtheta = torch.zeros(number_of_occupied_orbitals, number_of_parameters)
print(dnorm_wrt_dtheta.size())

torch.Size([5, 2])


## Derivative from Roothan equations: 

$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \vec{\theta}} =
\sum_j C_{aj}\int b_i(\vec{r}) \frac{\partial V_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}} b_j(\vec{r})d\vec{r}$$

As we know, $$V_{XC}[\rho](\vec{r};\;\vec{\theta}) = \frac{\partial E_{XC}[\rho]}{\partial\rho(\vec{r})} = \frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial\rho(\vec{r})}$$
So

$$\frac{\partial r_{ia}(\textbf{C};\;\vec{\theta})}{\partial \vec{\theta}} =
\sum_j C_{aj}\int b_i(\vec{r}) \frac{\partial\rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}\partial\rho(\vec{r})} b_j(\vec{r})d\vec{r} = 
\sum_j C_{aj}\int b_i(\vec{r}) \frac{ \frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}}{\partial\rho(\vec{r})} b_j(\vec{r})d\vec{r}
$$
It means, we can use $\frac{\partial \rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})}{\partial \vec{\theta}}$ instead of $\rho(\vec{r})\epsilon_{XC}[\rho](\vec{r};\;\vec{\theta})$ in DQC function`get_vxc()`and get suitable result. This function takes densinfo and takes functional derivative with respect to density.

In [63]:
dvxc_wrt_dtheta = adapter.get_derivative_of_vxc_wrt_theta()

In [64]:
dRoothan_wrt_dtheta = torch.einsum("ijt,ja->iat", dvxc_wrt_dtheta, coefficients)

# Concatenate derivatives with respect to parameters

In [68]:
dY_wrt_dtheta = dRoothan_wrt_dtheta.reshape(nao_norb_product, number_of_parameters)
print(dRoothan_wrt_dtheta.size(), "=>", dY_wrt_dtheta.size())
dY_wrt_dtheta = torch.cat((dY_wrt_dtheta, dnorm_wrt_dtheta), 0)
print(dY_wrt_dtheta.size())

torch.Size([13, 5, 2]) => torch.Size([65, 2])
torch.Size([70, 2])


Check it:

In [69]:
for t in range(number_of_parameters):
    for i in range(number_of_all_orbitals + 1):
        for a in range(number_of_occupied_orbitals):
            index = number_of_occupied_orbitals * i + a
            if (i < number_of_all_orbitals):
                assert(dY_wrt_dtheta[index][t] == dRoothan_wrt_dtheta[i][a][t]),(i,a,t)
            else:
                assert(dY_wrt_dtheta[index][t] == dnorm_wrt_dtheta[a][t]),(a,t)

# Total derivative:

In [72]:
print("partial derivative:", dE_wrt_dtheta)
print(torch.matmul(adjoint_vector, dY_wrt_dtheta))
total_dE_wrt_dtheta = dE_wrt_dtheta - torch.matmul(adjoint_vector, dY_wrt_dtheta)
print("full derivative:", total_dE_wrt_dtheta)

partial derivative: tensor([ 9.9156, 19.0426], dtype=torch.float64, grad_fn=<CatBackward0>)
tensor([9.3625e-15, 3.5054e-14], dtype=torch.float64,
       grad_fn=<SqueezeBackward3>)
full derivative: tensor([ 9.9156, 19.0426], dtype=torch.float64, grad_fn=<SubBackward0>)


In [10]:
def DQC_calculation_tensors(system, a, p):
    atomzs, atomposs = dqc.parse_moldesc(system)
    mol = dqc.Mol((atomzs, atomposs), basis="6-31G")
    myxc = MyLDAX(a, p)
    qc = dqc.KS(mol, xc=myxc).run()
    return qc

def DQC_calculation(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    return DQC_calculation_tensors(system, a, p)

def finite_difference(system, a_value, p_value, coeff=0.000001):
    y = DQC_calculation(system, a_value, p_value).energy().detach()
    da = a_value * coeff
    dp = p_value * coeff
    y_plus_dy_a = DQC_calculation(system, a_value+da, p_value)
    y_plus_dy_p = DQC_calculation(system, a_value, p_value+dp)
    dy_wrt_da = (y_plus_dy_a.energy().detach() - y) / da
    dy_wrt_dp = (y_plus_dy_p.energy().detach() - y) / dp
    return torch.tensor((dy_wrt_da, dy_wrt_dp))

def reverse_mode(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    ene = DQC_calculation_tensors(system, a, p).energy()
    grad_a, grad_p = torch.autograd.grad(ene, (a, p))
    print(ene.detach())
    return torch.tensor((grad_a, grad_p))

def adjoint_method(system, a_value, p_value):
    qc = DQC_calculation(system, a_value, p_value)
    adapter = evg.DQCAdapter(qc)
    return adapter.get_derivative()

In [73]:
reverse_mode(water, 1.0, 2.0)

tensor(-38.2050, dtype=torch.float64)


tensor([ 9.9156, 19.0426], dtype=torch.float64)

In [74]:
finite_difference(water, 1.0, 2.0)

tensor([ 9.9156, 19.0426], dtype=torch.float64)

In [25]:
adjoint_method(water, 1.0, 2.0)

partial g: tensor([ 9.9156, 19.0426], dtype=torch.float64)
adjoint: 15.576124348236934
partial f: 46.60346291417592
product: tensor([8.0394e-15, 3.9445e-14], dtype=torch.float64)


tensor([ 9.9156, 19.0426], dtype=torch.float64)

In [13]:
h2_system = "H 0 0 0; H 1.4 0 0"
reverse_mode(h2_system, 1.0, 2.0)

tensor(-0.4216, dtype=torch.float64)


tensor([ 0.0885, -0.2367], dtype=torch.float64)

In [16]:
finite_difference(h2_system, 1.0, 2.0)

tensor([ 0.0885, -0.2367], dtype=torch.float64)

In [26]:
adjoint_method(h2_system, 1.0, 2.0)

partial g: tensor([ 0.0885, -0.2367], dtype=torch.float64)
adjoint: 0.048828658997428526
partial f: 0.2578241139072411
product: tensor([-1.7153e-18,  2.5180e-18], dtype=torch.float64)


tensor([ 0.0885, -0.2367], dtype=torch.float64)

In [18]:
reverse_mode(h2_system, 44, 11)

tensor(-0.5196, dtype=torch.float64)


tensor([ 1.5659e-08, -1.0437e-06], dtype=torch.float64)

In [19]:
finite_difference(h2_system, 44, 11)

tensor([ 1.5654e-08, -1.0438e-06], dtype=torch.float64)

In [27]:
adjoint_method(h2_system, 44, 11)

partial g: tensor([ 1.5659e-08, -1.0437e-06], dtype=torch.float64)
adjoint: 0.18948747907059468
partial f: 1.0588627019790408e-05
product: tensor([ 3.5226e-23, -2.1861e-21], dtype=torch.float64)


tensor([ 1.5659e-08, -1.0437e-06], dtype=torch.float64)

In [21]:
n2_system = "N 0 0 0; N 2.07 0 0"
reverse_mode(n2_system, 1.0, 2.0)

The 6-31G basis for atomz 7 does not exist, but we will download it
Downloaded to /home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/dqc/api/.database/6-31g/07.gaussian94


/home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.810e-02, |f|=3.013e-01 at iter 39
  warnings.warn(ConvergenceWarning(msg))


tensor(-54.3529, dtype=torch.float64)


tensor([14.7435, 25.8682], dtype=torch.float64)

In [28]:
finite_difference(n2_system, 1.0, 2.0, 0.01)

/home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.810e-02, |f|=3.013e-01 at iter 39
  warnings.warn(ConvergenceWarning(msg))
/home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=2.862e-02, |f|=2.400e-01 at iter 31
  warnings.warn(ConvergenceWarning(msg))
/home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=1.865e-01, |f|=2.699e-01 at iter 46
  warnings.warn(ConvergenceWarning(msg))


tensor([13.0084, 24.6871], dtype=torch.float64)

In [29]:
adjoint_method(n2_system, 1.0, 2.0)

partial g: tensor([14.8463, 26.2175], dtype=torch.float64)
adjoint: 16.737336037940327
partial f: 46.10746032150164
product: tensor([2.9897e-14, 2.2423e-14], dtype=torch.float64)


tensor([14.8463, 26.2175], dtype=torch.float64)

In [30]:
ammonia = "N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721"
reverse_mode(ammonia, 1.0, 2.0)

tensor(-28.1712, dtype=torch.float64)


tensor([ 7.4649, 12.7271], dtype=torch.float64)

In [32]:
finite_difference(ammonia, 1.0, 2.0)

tensor([ 7.4649, 12.7271], dtype=torch.float64)

In [33]:
adjoint_method(ammonia, 1.0, 2.0)

partial g: tensor([ 7.4649, 12.7271], dtype=torch.float64)
adjoint: 11.980852386582916
partial f: 32.279433090828455
product: tensor([-4.7408e-15, -3.3333e-14], dtype=torch.float64)


tensor([ 7.4649, 12.7271], dtype=torch.float64)

In [34]:
benzene_coords = """C        0.598362921      0.000000000     -4.742986733;
                    C       -0.557705772     -0.354690359     -4.044822733;
                    C        1.754431614      0.354690359     -4.044822733;
                    H       -1.457130878     -0.630640582     -4.587995733;
                    H        2.653856720      0.630640582     -4.587995733;
                    C       -0.557705772     -0.354690359     -2.648492733;
                    C        1.754431614      0.354690359     -2.648492733;
                    H       -1.457130878     -0.630640582     -2.105319733;
                    H        2.653856720      0.630640582     -2.105319733;
                    C        0.598362921      0.000000000     -1.950328733;
                    H        0.598362921      0.000000000     -0.863981733;
                    H        0.598362921      0.000000000     -5.829333733"""
reverse_mode(benzene_coords, 1.0, 2.0)

The 6-31G basis for atomz 6 does not exist, but we will download it
Downloaded to /home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/dqc/api/.database/6-31g/06.gaussian94


/home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=6.229e-02, |f|=3.108e-01 at iter 39
  warnings.warn(ConvergenceWarning(msg))


tensor(-82.6662, dtype=torch.float64)


tensor([43.1028, 51.3126], dtype=torch.float64)

In [35]:
finite_difference(benzene_coords, 1.0, 2.0, 0.01)

/home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=1.024e-01, |f|=2.134e-01 at iter 49
  warnings.warn(ConvergenceWarning(msg))
/home/rolan/miniconda3/envs/noa-docs/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=4.371e-02, |f|=2.873e-01 at iter 34
  warnings.warn(ConvergenceWarning(msg))


tensor([42.4951, 50.1887], dtype=torch.float64)

In [ ]:
adjoint_method(benzene_coords, 1.0, 2.0)